In [1]:
# -*- coding:utf-8 -*-
# LDA算法实现
# 方法：在分好词的正面评价、负面评价以及过滤用的停用词表上进行，使用Gensim库完成LDA分析代码
import pandas as pd

# 参数初始化
inputfile1 = u'4_1my_meidi_jd_process_end_负面情感结果_cut.txt' 
inputfile2 = u'4_2my_meidi_jd_process_end_正面情感结果_cut.txt'
inputfile3 = 'stoplist.txt' # 停用词表

data1 = pd.read_csv(inputfile1, encoding = 'utf-8', header = None) #(***)
data2 = pd.read_csv(inputfile2, encoding = 'utf-8', header = None) #(***)
stop = pd.read_csv(inputfile3, encoding = 'utf-8', sep = 'tipdm', header = None) #(***)
# sep 设置分割词， 由于csv默认以半角逗号为分割词，而该词恰好在停用词表中，因此会导致读取出错
# 所以，解决方法是手动设置一个不存在的分割词，如tipdm
print data1.head()
print data2.head()
print stop.head()


                                                   0
0    好像 遥控 是 坏 的   还是 送 的 电池 没有 电   算了   热水器 上将 就...
1                         要 打 十个 字 才能 发   我 就 打 十个 字
2    调温 的 开关 太紧 了   不 知道 是不是 都 这样   送货 和 安装 的 师傅 ...
3           上面 安装 既然 花 了 我 差不多 * 块   但是 这 热水器 马马虎虎 吧
4             这 东西 有 不是 什么 高科技   比 别的 厂家 还贵   想 不 明白
                                                   0
0                                      还好   安装费 有点 贵
1    商品 已经 收到   打开 包装 检查一下 外观 完美 ； 还 没有 安装 使用   用...
2                             东西 不错   租房子 用 的   足够 了
3                            很 好   今天 安装 好 了   非常 满意
4                               可以 把   能用 就 好   出租 的
        0
0       说
1       人
2       元
3  hellip
4       &


D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


In [2]:
stop = [' ', ''] +list(stop[0]) # pandas自动过滤了空格符，所以手动将其加入

data1 [1] = data1[0].apply(lambda s: s.split(' ')) # 定义一个分隔函数，用apply广播
data1 [2] = data1[1].apply(lambda x: [i for i in x if i not in stop]) # 逐词判断是否是停用词

data2 [1] = data2[0].apply(lambda s: s.split(' ')) # 定义一个分隔函数，用apply广播
data2 [2] = data2[1].apply(lambda x: [i for i in x if i not in stop]) # 逐词判断是否是停用词
print data1.head()
print data2.head()


D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.
D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


                                                   0  \
0    好像 遥控 是 坏 的   还是 送 的 电池 没有 电   算了   热水器 上将 就...   
1                         要 打 十个 字 才能 发   我 就 打 十个 字   
2    调温 的 开关 太紧 了   不 知道 是不是 都 这样   送货 和 安装 的 师傅 ...   
3           上面 安装 既然 花 了 我 差不多 * 块   但是 这 热水器 马马虎虎 吧   
4             这 东西 有 不是 什么 高科技   比 别的 厂家 还贵   想 不 明白   

                                                   1  \
0  [, , 好像, 遥控, 是, 坏, 的, , , 还是, 送, 的, 电池, 没有, 电,...   
1       [, , 要, 打, 十个, 字, 才能, 发, , , 我, 就, 打, 十个, 字]   
2  [, , 调温, 的, 开关, 太紧, 了, , , 不, 知道, 是不是, 都, 这样, ...   
3  [, , 上面, 安装, 既然, 花, 了, 我, 差不多, *, 块, , , 但是, 这...   
4  [, , 这, 东西, 有, 不是, 什么, 高科技, , , 比, 别的, 厂家, 还贵,...   

                                                   2  
0  [好像, 遥控, 是, 坏, 的, 还是, 送, 的, 电池, 没有, 电, 算了, 热水器...  
1               [要, 打, 十个, 字, 才能, 发, 我, 就, 打, 十个, 字]  
2  [调温, 的, 开关, 太紧, 了, 不, 知道, 是不是, 都, 这样, 送货, 和, 安...  
3  [上面, 安装, 既然, 花, 了, 我, 差不多, 块, 但是, 这, 热水器, 马马虎虎...  
4   [这, 东西, 有, 不是, 什么, 高科技, 比, 别的, 厂家, 还贵, 想, 不, 明白

In [3]:
from gensim import corpora, models

D:\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.


In [4]:
# 负面主题分析
data1_dict = corpora.Dictionary(data1[2]) # 建立词典
data1_corpus = [data1_dict.doc2bow(i) for i in data1[2]] # 建立语料库

data1_LDA = models.LdaModel(data1_corpus, num_topics =3, id2word = data1_dict) # LDA训练模型
for i in range(3):
    data1_LDA.print_topic(i)# 输出每个主题

In [5]:
# 正面主题分析
data2_dict = corpora.Dictionary(data2[2]) # 建立词典
data2_corpus = [data2_dict.doc2bow(i) for i in data2[2]] # 建立语料库

data2_LDA = models.LdaModel(data2_corpus, num_topics =3, id2word = data2_dict) # LDA训练模型
for i in range(3):
    data2_LDA.print_topic(i)# 输出每个主题

In [8]:
data1_LDA.print_topic(0)

u'0.044*"\u7684" + 0.036*"\u8fd8" + 0.027*"\u5b89\u88c5" + 0.024*"\u4e86" + 0.021*"\u5f88" + 0.019*"\u4e0d\u9519" + 0.015*"\u4e0d" + 0.013*"\u7528" + 0.011*"\u597d" + 0.011*"\u5c31\u662f"'

In [9]:
data2_LDA.print_topic(1)

u'0.074*"\u5f88" + 0.072*"\u7684" + 0.058*"\u597d" + 0.032*"\u4e0d\u9519" + 0.030*"\u7528" + 0.030*"\u4e86" + 0.024*"\u4e70" + 0.017*"\u70ed\u6c34\u5668" + 0.012*"\u975e\u5e38" + 0.012*"\u4e5f"'